In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

from pathlib import Path

data_dir = "../artifacts/data"

In [ ]:
# Hyper-parameters 
input_size = 784
num_classes = 10
num_epochs = 50
batch_size = 100
learning_rate = 0.001

In [ ]:
# Preprocess Data
MNIST_train = torchvision.datasets.MNIST(data_dir, 
                                         train=True, 
                                         transform=transforms.ToTensor(),
                                         download=True)
MNIST_test = torchvision.datasets.MNIST(data_dir, 
                                        train=False, 
                                        transform=transforms.ToTensor(),
                                        download=True)

X = MNIST_train.data.reshape(-1, input_size)
y = MNIST_train.targets

In [ ]:
# Use sklearn logreg model
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(fit_intercept=True,
                        solver='saga',
                        max_iter=1000,
                        verbose=2,
                        n_jobs=5,
                        tol=0.01 # Tolerance for Stopping Criteria - we can keep it quite high as we just want to capture the hardest samples
                        )
clf

In [ ]:
# Fit the model (untill convergence)
clf.fit(X, y)

In [ ]:
# Sanity check - test on MNIST test
X_test = MNIST_test.data.reshape(-1, input_size)
y_test = MNIST_test.targets
score = clf.score(X_test, y_test) # test score
score

In [ ]:
# Find the hardest samples
# Compute per-sample loss (hardness)
probs = clf.predict_proba(X)
sample_losses = -np.log(probs[np.arange(len(y)), y])

# Find top 100 hardest per class
hardest_per_class = {}
for c in range(10):
    class_indices = np.where(y == c)[0]
    sorted_idx = class_indices[np.argsort(sample_losses[class_indices])[::-1]]
    hardest_per_class[c] = sorted_idx[:100]  # top 100 hardest

# Example: hardest indices for class 3
print("Hardest samples for class 3:", hardest_per_class[3][:10])

In [ ]:
# Visualise